In [1]:
import anndata as ad
import numpy as np
import scanpy as sc
import json

# unpaired

In [2]:
adata = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/multimap_trimodal_integrated.h5ad')
adata

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 147902 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities'

In [3]:
with open('/lustre/groups/ml01/workspace/anastasia.litinetskaya/trimodal_suppl_benchmarks_data/trimodal/cellttype_harmonize.json', 'r') as f:
    harmonized_celltypes = json.load(f)
harmonized_celltypes.keys()

dict_keys(['cite_ct_l1_map', 'cite_ct_l2_map', 'multi_ct_l1_map', 'multi_ct_l2_map'])

In [4]:
adata.obs['Modality']

TAGTTGTCACCCTCAC-1-s1d1         multiome
CTATGGCCATAACGGG-1-s1d1         multiome
CCGCACACAGGTTAAA-1-s1d1         multiome
TCATTTGGTAATGGAA-1-s1d1         multiome
ACCACATAGGTGTCCA-1-s1d1         multiome
                               ...      
TCGTGCTTCGTTACAA-6-s2d5    multiome_atac
TGTGGAGCAGGTTAAA-6-s2d5    multiome_atac
TTCGCAACATGATTGT-6-s2d5    multiome_atac
AAGTAGCCATGTGGGA-6-s2d5    multiome_atac
CTCACTCAGCCTTAAA-6-s2d5    multiome_atac
Name: Modality, Length: 147902, dtype: category
Categories (4, object): ['cite', 'cite_adt', 'multiome', 'multiome_atac']

In [5]:
adata.obs['cell_type'][adata.obs['Modality'].isin(['cite', 'cite_adt'])]

GCATTAGCATAAGCGG-1-s1d1    Naive CD20+ B IGKC+
TACAGGTGTTAGAGTA-1-s1d1             CD14+ Mono
AGGATCTAGGTCTACT-1-s1d1    Naive CD20+ B IGKC+
GTAGAAAGTGACACAG-1-s1d1                    HSC
TCCGAAAAGGATCATA-1-s1d1           Reticulocyte
                                  ...         
TCCGGGAGTTGGGTTT-1-s2d5                     NK
TTGGATGAGGTGCCAA-1-s2d5            NK CD158e1+
ACAGAAACACAGCGCT-1-s2d5    Naive CD20+ B IGKC+
CACACAAGTCTAATCG-1-s2d5           CD4+ T naive
GTCCACTAGCTACGTT-1-s2d5           CD4+ T naive
Name: cell_type, Length: 82964, dtype: category
Categories (48, object): ['B1 B', 'B1 B IGKC+', 'B1 B IGKC-', 'CD4+ T CD314+ CD45RA+', ..., 'dnT', 'gdT CD158b+', 'gdT TCRVD2+', 'pDC']

In [6]:
adata.obs['cell_type'][adata.obs['Modality'].isin(['multiome', 'multiome_atac'])]

TAGTTGTCACCCTCAC-1-s1d1    Naive CD20+ B
CTATGGCCATAACGGG-1-s1d1       CD14+ Mono
CCGCACACAGGTTAAA-1-s1d1           CD8+ T
TCATTTGGTAATGGAA-1-s1d1           CD8+ T
ACCACATAGGTGTCCA-1-s1d1       CD16+ Mono
                               ...      
TCGTGCTTCGTTACAA-6-s2d5               NK
TGTGGAGCAGGTTAAA-6-s2d5       CD16+ Mono
TTCGCAACATGATTGT-6-s2d5           CD8+ T
AAGTAGCCATGTGGGA-6-s2d5               NK
CTCACTCAGCCTTAAA-6-s2d5       CD14+ Mono
Name: cell_type, Length: 64938, dtype: category
Categories (48, object): ['B1 B', 'B1 B IGKC+', 'B1 B IGKC-', 'CD4+ T CD314+ CD45RA+', ..., 'dnT', 'gdT CD158b+', 'gdT TCRVD2+', 'pDC']

In [7]:
adata.obs['l2_cell_type'] = 'test'
multiome_mask = adata.obs['Modality'].isin(['multiome', 'multiome_atac'])
cite_mask = adata.obs['Modality'].isin(['cite', 'cite_adt'])
adata.obs.loc[multiome_mask, 'l2_cell_type'] = adata.obs['cell_type'].loc[multiome_mask].map(harmonized_celltypes['multi_ct_l2_map'])
adata.obs.loc[cite_mask, 'l2_cell_type'] = adata.obs['cell_type'].loc[cite_mask].map(harmonized_celltypes['cite_ct_l2_map'])

In [8]:
adata.obs['l2_cell_type'][multiome_mask]

TAGTTGTCACCCTCAC-1-s1d1       Naive CD20+ B
CTATGGCCATAACGGG-1-s1d1          CD14+ Mono
CCGCACACAGGTTAAA-1-s1d1    CD8+ T activated
TCATTTGGTAATGGAA-1-s1d1    CD8+ T activated
ACCACATAGGTGTCCA-1-s1d1          CD16+ Mono
                                 ...       
TCGTGCTTCGTTACAA-6-s2d5                  NK
TGTGGAGCAGGTTAAA-6-s2d5          CD16+ Mono
TTCGCAACATGATTGT-6-s2d5    CD8+ T activated
AAGTAGCCATGTGGGA-6-s2d5                  NK
CTCACTCAGCCTTAAA-6-s2d5          CD14+ Mono
Name: l2_cell_type, Length: 64938, dtype: object

In [9]:
adata.obs['l2_cell_type'][cite_mask]

GCATTAGCATAAGCGG-1-s1d1    Naive CD20+ B
TACAGGTGTTAGAGTA-1-s1d1       CD14+ Mono
AGGATCTAGGTCTACT-1-s1d1    Naive CD20+ B
GTAGAAAGTGACACAG-1-s1d1              HSC
TCCGAAAAGGATCATA-1-s1d1     Reticulocyte
                               ...      
TCCGGGAGTTGGGTTT-1-s2d5               NK
TTGGATGAGGTGCCAA-1-s2d5               NK
ACAGAAACACAGCGCT-1-s2d5    Naive CD20+ B
CACACAAGTCTAATCG-1-s2d5     CD4+ T naive
GTCCACTAGCTACGTT-1-s2d5     CD4+ T naive
Name: l2_cell_type, Length: 82964, dtype: object

In [10]:
adata.obsm['latent'] = adata.obsm['X_multimap'].copy()

In [11]:
adata

AnnData object with n_obs × n_vars = 147902 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap', 'latent'
    layers: 'counts'
    obsp: 'connectivities'

In [12]:
del adata.layers['counts']
adata.write('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/multimap_trimodal_integrated_clean.h5ad')

# average of the embeddings for each cell

In [13]:
adata = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/multimap_trimodal_integrated_clean.h5ad')
adata

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 147902 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap', 'latent'
    obsp: 'connectivities'

In [14]:
adata.obs['Modality'].cat.categories

Index(['cite', 'cite_adt', 'multiome', 'multiome_atac'], dtype='object')

In [15]:
rna_multiome = adata[adata.obs['Modality'] == 'multiome'].copy()
rna_multiome

AnnData object with n_obs × n_vars = 32469 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap', 'latent'
    obsp: 'connectivities'

In [16]:
atac_multiome = adata[adata.obs['Modality'] == 'multiome_atac'].copy()
atac_multiome

AnnData object with n_obs × n_vars = 32469 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap', 'latent'
    obsp: 'connectivities'

In [17]:
np.sum(rna_multiome.obs_names != atac_multiome.obs_names)

0

In [18]:
rna_cite = adata[adata.obs['Modality'] == 'cite'].copy()
rna_cite

AnnData object with n_obs × n_vars = 41482 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap', 'latent'
    obsp: 'connectivities'

In [19]:
adt_cite = adata[adata.obs['Modality'] == 'cite_adt'].copy()
adt_cite

AnnData object with n_obs × n_vars = 41482 × 32849
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    uns: 'neighbors'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_lsi', 'X_multimap', 'X_pca', 'X_umap', 'latent'
    obsp: 'connectivities'

In [20]:
np.sum(rna_cite.obs_names != adt_cite.obs_names)

0

In [21]:
multiome = sc.AnnData(X = 0.5*(rna_multiome.obsm['latent'] + atac_multiome.obsm['latent']), obs=rna_multiome.obs)
multiome

AnnData object with n_obs × n_vars = 32469 × 2
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'

In [22]:
multiome.obsm['latent'] = multiome.X.copy()

In [23]:
cite = sc.AnnData(X = 0.5*(rna_cite.obsm['latent'] + adt_cite.obsm['latent']), obs=rna_cite.obs)
cite

AnnData object with n_obs × n_vars = 41482 × 2
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'

In [24]:
cite.obsm['latent'] = cite.X.copy()

In [25]:
latent = ad.concat([multiome, cite])
latent

AnnData object with n_obs × n_vars = 73951 × 2
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'new_batch', 'GEX_n_genes_by_counts', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'ADT_pseudotime_order', 'is_train', 'concat_key', 'l2_cell_type'
    obsm: 'latent'

In [26]:
latent.write('/lustre/groups/ml01/workspace/anastasia.litinetskaya/notebooks/snakemake/trimodal/data/multimap_trimodal_integrated_clean_average.h5ad')